# IDRNN
[Description]
(Add description here)

The goal of the project:
1. palceholder
2. placeholder


# Dataset

First download the dataset.

# Packages

In [ ]:
# !pip3 install -r requirements.txt

In [4]:
%load_ext autoreload
%autoreload 2
import numpy as np
import torch
import tqdm
from torch.utils.data import  Dataset, DataLoader
import torch.nn.functional as F
from sklearn.model_selection import train_test_split


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# load data and print it

In [5]:
from dataloader import ProteinDataset, proteinread

In [8]:
# Replace this path with the actual path to your input CSV file
from sklearn.model_selection import train_test_split
input_csv_file = "data.csv"

# Create an instance of the ProteinDataset
dataset = proteinread(input_csv_file)

# Define the validation set size (20% of the total data)
training_size = 0.8

# Split the dataset into training and validation sets
train_data, val_data = train_test_split(dataset, train_size=training_size, random_state=42)

# Create dataset instances for training and validation
train_data, val_data = ProteinDataset(train_data), ProteinDataset(val_data)

# Create data loaders for training and validation
batch_size = 64  # You can adjust this batch size
train_dataset = DataLoader(train_data, batch_size=batch_size, shuffle=True)
val_dataset = DataLoader(val_data, batch_size=batch_size, shuffle=False)


ValueError: expected sequence of length 490 at dim 1 (got 491)

In [ ]:
# pick up one sequence to show how our data looks like
g = training_dataset[65]

print(g)


In [ ]:
# to get training sequence and validation sequence only


# Create an empty list to store the numerical sequences
training_sequences = []
validation_sequences = []
# Iterate over the training_dataset and  validation dataset
for sample in training_dataset:
    # Extract the numerical part (the first element of the tuple) and append it to the list
    training_sequences.append(sample[0])

for sample in validation_dataset:
    # Extract the numerical part (the first element of the tuple) and append it to the list
    validation_sequences.append(sample[0])
    
# Print the list of training sequences
print(training_sequences[65])

In [ ]:
print(training_sequences)

# Helper function and calculation (kld reconstruct loss) and printer form tuorial 6

In [ ]:
## Help function from tutroial 6 to calculate loss and plotting

def sample(mean, log_var):
    std = torch.exp(0.5*log_var)
    eps = torch.randn_like(std, device=mean.device)
    return mean + eps*std
def count_parameters(model):
    total_params = sum(p.numel() for p in model.parameters())
    return total_params
def plot_losses(reco_losses, kld_losses):
    fig = plt.figure(figsize=(10, 5))

    ax = fig.add_subplot(1, 2, 1)
    ax.plot(reco_losses)
    ax.set_title('Reconstruction Loss')

    ax = fig.add_subplot(1, 2, 2)
    ax.plot(kld_losses)
    ax.set_title('KLD Loss')
def generate_imgs(decoder_nn, cond=False):
    num_img = 32
    new_z = torch.randn(32, 2)
    if torch.cuda.is_available():
        new_z = new_z.cuda()

    if cond:
        new_labels = torch.zeros(num_img, 10).cuda()
        rand_int = torch.randint(0, 10, (num_img,)).cuda()
        new_labels[torch.arange(num_img), rand_int] = 1

        new_pred = decoder_nn(new_z, new_labels)
    else:
        new_pred = decoder_nn(new_z)

    new_pred = new_pred.cpu().detach().numpy()

    fig = plt.figure(figsize=(20, 10))
    gs = GridSpec(4, 8)

    for i in range(32):
        ax = fig.add_subplot(gs[i])
        img_array = new_pred[i][0]
        ax.imshow(img_array, cmap='gray')
        ax.axis('off')
        if cond:
            ax.set_title(f'Cond input: {rand_int[i]}')
def plot_z_dist(train_dl, encoder_nn, cond=False):
    fig = plt.figure(figsize=(24, 6))
    gs = GridSpec(1, 4)


    mus = []; log_vars = []; zs = []; ys = []
    for xb, yb, yb_one_hot in train_dl:
        if torch.cuda.is_available():
            xb = xb.cuda()
            yb_one_hot = yb_one_hot.cuda()

        if cond:
            mu, log_var = encoder_nn(xb, yb_one_hot)
        else:
            mu, log_var = encoder_nn(xb)
        z = sample(mu, log_var)

        mu = mu.cpu().detach().numpy()
        log_var = log_var.cpu().detach().numpy()
        z = z.cpu().detach().numpy()

        mus.extend(mu.tolist())
        log_vars.extend(log_var.tolist())
        zs.extend(z.tolist())
        ys.extend(yb.tolist())

    zs = np.array(zs)
    mus = np.array(mus)
    log_vars = np.array(log_vars)
    ys = np.array(ys)


    ax = fig.add_subplot(gs[0])
    ax.scatter(zs[:, 0], zs[:, 1], c=ys, cmap='tab10', alpha=0.3, s=1)
    ax.set_title('Z')
    ax.grid()
    ax.set_aspect('equal')

    gauss0 = np.random.normal(0, 1, len(zs[:, 0]))
    gauss1 = np.random.normal(0, 1, len(zs[:, 0]))
    ax = fig.add_subplot(gs[1])
    ax.scatter(gauss0, gauss1, c='cornflowerblue', alpha=0.3, s=1)
    ax.set_title('Gaus(0,1))')
    ax.set_aspect('equal')

    return ys, zs

# trainning loop based on tutorial 6

In [ ]:
from model_VAE import EncoderLn, DecoderLn
encoder_nn = EncoderLn()
decoder_nn = DecoderLn()

if torch.cuda.is_available():
    encoder_nn.cuda()
    decoder_nn.cuda()

In [ ]:

optimizer = torch.optim.Adam(list(encoder_nn.parameters()) + list(decoder_nn.parameters()), lr=0.001)


reco_losses = []
kld_losses  = []

for ep in tqdm(range(50)):
    for x _ in train_dl:
        if torch.cuda.is_available():
            x = x.cuda()


        optimizer.zero_grad()

        mu, log_var = encoder_nn(xb)
        z = sample(mu, log_var)

        pred = decoder_nn(z)

        # loss
        reco_loss = BCE = F.binary_cross_entropy(pred, x, reduction='sum')
        kld_loss  = -0.5 * torch.sum(1 + log_var - mu**2 -  log_var.exp())

        loss = reco_loss + kld_loss

        loss.backward()
        optimizer.step()

        reco_losses.append(reco_loss.item())
        kld_losses.append(kld_loss.item())
